# Modifying an image in a project
In this notebook, we will show how to take a single (annotated) image from a project, and modify it by converting it to grayscale. 

True in-place modification of images in the project is not possible, so we will take a step by step approach: First we will download the image, then convert it to grayscale on our local machine and then upload it again. Finally, we will have to take the annotation from the original image and apply it to the grayscale image we created. 

In [ ]:
# As usual we will connect to the platform first, using the server details from the .env file

from geti_sdk import Geti
from geti_sdk.rest_clients import ProjectClient
from geti_sdk.utils import get_server_details_from_env

geti_server_configuration = get_server_details_from_env()

geti = Geti(server_config=geti_server_configuration)

project_client = ProjectClient(session=geti.session, workspace_id=geti.workspace_id)

## Getting a project
First, let's get a project to work with. You can either create a new, COCO-based project with a few images and annotations, or select an existing project from the workspace.

#### Option 1: Creating a new project
Execute the next two code cells if you would like to create a new project

In [ ]:
from geti_sdk.demos import get_coco_dataset

COCO_PATH = get_coco_dataset()

In [ ]:
from geti_sdk.annotation_readers import DatumAnnotationReader

# Create the annotation reader and select the images with 'horse' in them
annotation_reader = DatumAnnotationReader(
    base_data_folder=COCO_PATH, annotation_format="coco"
)
annotation_reader.filter_dataset(labels=["horse"])

# Create the project, upload images and annotations
project = geti.create_single_task_project_from_dataset(
    project_name="COCO horse detection demo",
    project_type="detection",
    path_to_images=COCO_PATH,
    annotation_reader=annotation_reader,
    number_of_images_to_upload=12,
    number_of_images_to_annotate=12,
    enable_auto_train=True,
)

#### Option 2: Using an existing project 
If you prefer to use an existing project, the following cell will list all projects in the current workspace so that you can select one

In [ ]:
projects = project_client.list_projects()

Change the `project_name` here to select your project of choice

In [ ]:
project = project_client.get_project(project_name="COCO horse detection demo")

## Getting an image 
Now that we have a project we can get an image to modify. Let's get a list of all images in the project, and select the first one in the list as our target image

In [ ]:
from geti_sdk.rest_clients import ImageClient

image_client = ImageClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

images = image_client.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images")

image = images[0]

## Converting to grayscale
To convert the image to grayscale, we first have to fetch the pixel data from the platform. This can be done using the `image.get_data()` method. This method takes the `client.session` attribute as its argument, since this is used to connect to the platform.

In the cell below we fetch the image data and convert it to grayscale. At the end, IPython's `display.display` method is used to show the resulting image. This will show the image below the code cell.

In [ ]:
import cv2
from IPython.display import display
from PIL import Image as PILImage

image_numpy = image.get_data(session=geti.session)

# Convert numpy data to grayscale
grayscale_numpy = cv2.cvtColor(image_numpy, cv2.COLOR_RGB2GRAY)

display_image = PILImage.fromarray(grayscale_numpy)
display(display_image)

#### Uploading the grayscale image
Let's upload the new image to the project

In [ ]:
grayscale_image = image_client.upload_image(image=grayscale_numpy)

# Check that the number of images in the project increases by one when we do this
images = image_client.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images")

## Annotating the image
Great, we have converted the image to grayscale and uploaded it to the project. However, currently it is still missing an annotation. Since the grayscale conversion of course did not affect the objects in the image, we can just apply the annotation from the original image to the grayscale image directly. 

To do so, we first need to set up an AnnotationClient for the project and retrieve the original annotation

In [ ]:
from geti_sdk import Visualizer
from geti_sdk.rest_clients import AnnotationClient

annotation_client = AnnotationClient(
    session=geti.session, workspace_id=geti.workspace_id, project=project
)

# Retrieve the annotation for the original image
annotation = annotation_client.get_annotation(media_item=image)

# Inspect the annotation
print(annotation.overview)

visualizer = Visualizer()
image_rgb = cv2.cvtColor(image.numpy, cv2.COLOR_BGR2RGB)
result = visualizer.draw(image_rgb, annotation)
visualizer.show_in_notebook(result)

Let's try getting the annotation for the grayscale image. It should not return anything, since we have not annotated it yet

In [ ]:
grayscale_annotation = annotation_client.get_annotation(media_item=grayscale_image)
print(grayscale_annotation)

#### Applying the annotation
Alright, now we are all set to annotate the grayscale image. Note that we passed `media_item=grayscale_image` to the `get_annotation` method in the cell above. It is important to use the `grayscale_image` rather than `grayscale_numpy`, because the grayscale_image holds the actual reference to the image on the platform; grayscale_numpy is just the pixel data, and the annotation_client would not know how to handle it.

In [ ]:
grayscale_annotation = annotation_client.upload_annotation(
    media_item=grayscale_image, annotation_scene=annotation
)

# Inspect the annotation
result = visualizer.draw(grayscale_image.get_data(geti.session), grayscale_annotation)
visualizer.show_in_notebook(result)

That's it! To make sure that the annotation was indeed uploaded for the correct image, let's try to get it again using the `get_annotation` method from the annotation_client

In [ ]:
grayscale_annotation = annotation_client.get_annotation(media_item=grayscale_image)
print(grayscale_annotation.overview)

## Deleting the original image
This last step is optional: It deletes the RGB image from the project so that only the grayscale variant remains.

In [ ]:
image_client.delete_images([image])